In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import time

In [2]:
from tensorflow import keras

In [3]:
LABEL_DEMENSIONS = 10

In [4]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.fashion_mnist.load_data()

In [5]:
train_images = np.asarray(train_images, dtype='float32') / 255

In [6]:
train_images = train_images.reshape(-1, 28, 28, 1)
test_images = np.asarray(test_images, dtype='float32') / 255

In [7]:
test_images = test_images.reshape(-1, 28, 28, 1)

In [8]:
train_labels = keras.utils.to_categorical(train_labels, LABEL_DEMENSIONS)
test_labels = keras.utils.to_categorical(test_labels, LABEL_DEMENSIONS)

In [9]:
train_labels = train_labels.astype(np.float32)
test_labels = test_labels.astype(np.float32)

In [10]:
print(train_labels.shape, test_labels.shape)

(60000, 10) (10000, 10)


In [11]:
inputs = keras.Input(shape=(28, 28, 1))
x = keras.layers.Conv2D(filters = 32, kernel_size=(3, 3), activation='relu')(inputs)
x = keras.layers.MaxPooling2D(pool_size=(2, 2), strides = 2)(x)
x = keras.layers.Conv2D(filters = 64, kernel_size=(3, 3), activation='relu')(x)
x = keras.layers.MaxPooling2D(pool_size=(2, 2), strides = 2)(x)
x = keras.layers.Conv2D(filters = 64, kernel_size=(3, 3), activation='relu')(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, activation='relu')(x)

In [12]:
predictions = keras.layers.Dense(LABEL_DEMENSIONS, activation = 'softmax')(x)

In [13]:
model = keras.Model(inputs = inputs, outputs = predictions)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)              

In [14]:
optimizer = keras.optimizers.SGD()
model.compile(loss='categorical_crossentropy', optimizer = optimizer, metrics=['accuracy'])

In [15]:
# strategy = None
strategy = tf.distribute.MirroredStrategy()
config = tf.estimator.RunConfig(train_distribute=strategy)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Initializing RunConfig with distribution strategies.
INFO:tensorflow:Not using Distribute Coordinator.


In [16]:
estimator = keras.estimator.model_to_estimator(model, config=config)

INFO:tensorflow:Using the Keras model provided.
Instructions for updating:
Simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpiy6ppu04', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': <tensorflow.python.distribute.mirrored_strategy.MirroredStrategy object at 0x7f23d39136d0>, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 

In [17]:
def input_fn(images, labels, epochs, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    
    SHUFFLE_SIZE = 5000
    
    dataset = dataset.shuffle(SHUFFLE_SIZE).repeat(epochs).batch(batch_size)
    dataset = dataset.prefetch(None)
    
    return dataset

In [18]:
BATCH_SIZE = 512
EPOCHS = 50

estimator_train_result = estimator.train(input_fn = lambda:input_fn(train_images, train_labels, epochs=EPOCHS, batch_size=BATCH_SIZE))
print(estimator_train_result)

Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/devic

In [19]:
estimator.evaluate(lambda:input_fn(test_images, test_labels, 1, BATCH_SIZE))

INFO:tensorflow:Calling model_fn.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-08-09T14:12:12Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpiy6ppu04/model.ckpt-5860
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.25153s
INFO:tensorflow:Finished evaluation at 2020-08-09-14:12:13
INFO:tensorflow:Saving dict for global step 5860: accuracy = 0.8395, global_step = 5860, loss = 0.4414285
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5860: /tmp/tmpiy6ppu04/model.ckpt-5860


{'accuracy': 0.8395, 'loss': 0.4414285, 'global_step': 5860}